# B - Change date detection
**REQUIREMENT : (A) and UPDATED SAR_INDEX_STATS TABLE**

This program carries out change date detection using the following function:

>**change_date_detection(sites, dteDebut, dteFin, dteExe)**

For each SAR, this function operates the following steps:

**1) Preparation of dataframes as input for change date detection**

Dataframes consists of 4-year history of VH_[orbit] (3 different VH orbits by site) and NDWI2 (for each site).
The 4-year history is defined be the user using dteDebut and dteFin parameters.

**2) Change date detection** 
The module of change detection of ERM is used namely
> **change_detection(profiles[,day_end])**

>>Parameters:

>>>profiles: list of pandas DataFrames

>>>>All the (at least three) VH and NDWI2 temporal profiles available for a given site.

>>>day_end: string (optional)

>>>>Last day of analysis. Default is the day on which the routine is called. Format YYYY-MM-DD.

>>Returns:

>>>pandas DataFrame (change dates in [day_end-1Y, day_end-1M]) or 0 (if no change detection was performed)

>>>pandas Dataframe (VH feature in [day_end-4Y, day_end]) or 0 (if no VH feature is available)

Dataframes generated in 1) are provided as input.

Dataframes are processed by the module namely VH orbits are combined into one single VH signal. Gaps in VH and NDWI2 are filled and the resulted time-series are smoothed using a gaussian filter.

Change date are then using the smoothed VH and NDWI2 4-year time series calculated.

Change date are returned for the last year of the 4-year period minus one month .

The date (dteChgEnd = dteFin -1 month)

**3) Output are saved in tables**

Change dates are saved in a table (one table/SAR)

Recombined smoothed VH profile is saved in a table (one table/SAR)
Output tables are named as following:

'[(id_segment)_VH_(dteExe)_smoothed]'

'[(id_segment)_dates_(dteExe)]'


## Import libraries and functions

In [ ]:
import os, sys, datetime, pandas as pd
sys.path.append("/home/gswinnen/SARSAR_Package_RenPri/code/") # Localisation of SARSAR libraries
sys.path.append("/home/gswinnen/SARSAR_Mattia/")              # Localisation of change_detection module

#sys.path.append("/home/issep/sarsar-issep/SARSAR_utils/")                   # emplacement des modules RenPri
#sys.path.append("/home/issep/sarsar-issep/SARSAR_utils/rme_chg_detection_module/") # emplacement de la fonction de Mattia

import changedetection
from issep import sarsar_admin
from os.path import join
from lecture_ini import config
from select_sites import sites_to_process

## Definition of the **change_date_detection** function

In [ ]:
def change_date_detection(sites, dteDebut, dteFin, dteExe):
    
    """ This function operates change date detection.
        It uses 4-year VH_[orbit] and NDWI profiles to detect eventual change date over the last year of the considered 4-year period (dteDebut - dteFin).
        The function adds tables to the DB containing change dates and VH recombined and smoothed profiles.
    
            Parameters (they are automatically read in "sarsar.ini" by the function "config"):
                sites (list): list of sites (sar_id_segment) to process
                dteDebut (date): date from which we need data (YYYY-MM-DD)
                dteFin (date): date until which we need data (YYYY-MM-DD)
                dteExe: processing/execution date (YYYYMMDD)
            """
    
    ## CONNECTION TO DB
    # Define Database connection parameters
    # NOTE: password is in ~/.pgpass

    credentials = config(section='postgresql')
    db_credentials = {
        'host': credentials['host'],
        'user': credentials['user'],
        'db' : credentials['database']
    }

    # ALWAYS prepare env at the beginning
    print('> Preparing env (DB credentials, etc)')
    sarsar_admin.prepare_env(db_credentials)
    
    conn = sarsar_admin._create_or_get_db_connection()
    cur = None
    cur2 = None
    
    ## STARTING CHANGE DATE DETECTION

    try:
        import psycopg2.extras #allow communication with pgsql
        cur = conn.cursor(cursor_factory = psycopg2.extras.DictCursor)
        cur2 = conn.cursor(cursor_factory = psycopg2.extras.DictCursor)

        # Liste les sar_id_segment pour lesquels j'ai des observations dans l'intervalle de dates
#         strSQL = '''SELECT DISTINCT sar_id_segment 
#                     FROM sar_index_stats WHERE index_name NOT IN ('BI2','VV','SBI','NDVI','BI2_part1','BI','BAI','VH') 
#                     AND substring(index_name,1,2) != 'VV' AND acq_date BETWEEN '{0}' AND '{1}' 
#                     ORDER BY sar_id_segment;'''.format(dteDebut, dteFin)
#         cur.execute(strSQL)
#         sites = [item[0] for item in cur.fetchall()]

        # DEBUG: force la liste
    #        sites = ['62003-ISA-0007-01', '62063-ISA-0073-01', '62096-ISA-0056-01','52012-ISA-0010-01']
#        print("DEBUG: {0} sites trouvés".format(len(sites)))

#        i_debug = 438             # DEBUG
#        sites = sites[i_debug:]   # DEBUG

    ## PREPARATION OF INPUTS FOR CHANGE DETECTION
        for site in sites:
#            i_debug += 1
#            print(site, i_debug)  # DEBUG
            dataframes = []
            nomDataframe = ''
            
            # Selection of VH_"orbit" and NDWI 4-year profiles in the SAR_index_stats table
            strSQL = '''SELECT sar_id_segment, index_name, acq_date, index_mean, pixel_count 
                        FROM sar_index_stats 
                        WHERE sar_id_segment = '{0}' AND index_name NOT IN ('BI2','VV','SBI','NDVI','BI2_part1','BI','BAI','VH') 
                        AND substring(index_name,1,2) != 'VV' AND acq_date BETWEEN '{1}' AND '{2}' 
                        ORDER BY sar_id_segment, index_name, acq_date;'''.format(site, dteDebut, dteFin)
            cur.execute(strSQL)
            result = cur.fetchall()

            # Sequentially scans "result"
            # For each new sar_id_segment/index_name, a pandas dataframe is created
            # For a given site (sar_id_segment), the name of the created df are registered in a list
            for row in result:
                if nomDataframe not in ['{0}_S1_{1}_profile'.format(row['sar_id_segment'], row['index_name']), '{0}_S2_{1}_profile'.format(row['sar_id_segment'], row['index_name'])] :
                    if nomDataframe != '':
                        # If it exists, the current dataframe is registered in the list
                        dataframes.append(df)

                    # Creation of new df for VH_"orbit" indexes
                    # NB: the df exist only within the execution of the program
                    if row['index_name'] != 'NDWI2':
                        nomDataframe = '{0}_S1_{1}_profile'.format(row['sar_id_segment'], row['index_name'])
                        colDataframe = ['Date', 'Hour', row['index_name'].replace('VH', 'VH_'), 'Size']

                        df = pd.DataFrame([(row['acq_date'].strftime("%Y-%m-%d"), '06:00:00', row['index_mean'], row['pixel_count'])], columns = colDataframe)
                    else:
                        # Creation of new df for NDWI2 indexes
                        # Do not process NDWI2 whose average index is 0 (outliers)
                        if row['index_mean'] != 0:
                            nomDataframe = '{0}_S2_{1}_profile'.format(row['sar_id_segment'], row['index_name'])
                            colDataframe = ['Date', row['index_name']]

                            df = pd.DataFrame([(row['acq_date'].strftime("%Y-%m-%d"), row['index_mean'])], columns = colDataframe)
                    # For a given site (sar_id_segment), the name of the created df are registered in a list
                    df.name = nomDataframe

                else:
                    # Add line to the current df
                    if row['index_name'] != 'NDWI2':
                        df = df.append({'Date':row['acq_date'].strftime("%Y-%m-%d"), 'Hour':'06:00:00', row['index_name'].replace('VH', 'VH_'):row['index_mean'], 'Size':row['pixel_count']} , ignore_index=True)
                    else:
                        df = df.append({'Date':row['acq_date'].strftime("%Y-%m-%d"), row['index_name']:row['index_mean']} , ignore_index=True)

            # Add the last df to the list of df available for the current site
            dataframes.append(df)
            

    ## CHANGE DETECTION
            # The change detection module is called with the list of dataframes and dteFin as inputs
            results = changedetection.change_detection(dataframes, dteFin)

    ## SAVING CHANGE DETECTION OUTPUTS IN TABLES
            # The output (df) of the change detection are stored in tables
            
            # First, the recombined and smoothed VH profile is saved
            if isinstance(results[1], pd.DataFrame):                       # We check if the change detection has returned a df
                table_name = '{0}_VH_{1}_smoothed'.format(site, dteExe)
                strSQL = 'DROP TABLE IF EXISTS "{0}" CASCADE;'.format(table_name)  # Surround the table name to tolerate numbers at the beginning and dashes in the middle
                cur2.execute(strSQL)
                conn.commit()

                strSQL = 'CREATE TABLE IF NOT EXISTS "{0}" (dte DATE, indice NUMERIC);'.format(table_name)  #, ', '.join(type_valeurs))
                cur2.execute(strSQL)

                for index, row in results[1].iterrows():
                    strSQL = 'INSERT INTO "{0}" (dte, indice) VALUES (\'{1}\', {2});'.format(table_name, index, row['VH'])  # 'Date' is the df index
            #        print(strSQL)
                    cur2.execute(strSQL)

                # Post all modifications
                conn.commit()
            else:
                print("DEBUG: results[1] == 0")

            # Second, the change date(s) are saved (same logic)
            table_name = '{0}_dates_{1}'.format(site, dteExe)
            strSQL = 'DROP TABLE IF EXISTS "{0}" CASCADE;'.format(table_name) 
            cur2.execute(strSQL)
            conn.commit()

            strSQL = 'CREATE TABLE IF NOT EXISTS "{0}" (Change_date DATE);'.format(table_name)  #, ', '.join(type_valeurs))
            cur2.execute(strSQL)

            if isinstance(results[0], pd.DataFrame):
                for index, row in results[0].iterrows():
                    strSQL = 'INSERT INTO "{0}" (Change_date) VALUES (\'{1}\');'.format(table_name, index)  
                    print(strSQL)
                    cur2.execute(strSQL)

                conn.commit()
            else:
                print("DEBUG: results[0] == 0")
            
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if cur2 is not None:
            cur2.close()
        if cur is not None:
            cur.close()

    # ALWAYS release env at the end
    print('> Releasing env')
    sarsar_admin.release_env()

## Calling functions

In [ ]:
## Call config
dates = config(section='dates')
dteDebut = dates['deb']
dteFin = dates['fin']
dteExe = dates['exe']

## Call sites_to_process
lstSARs = sites_to_process(dteDebut, dteFin)

## Call change_detection
change_date_detection(lstSARs, dteDebut, dteFin, dteExe)